# 세이지메이커 점프스타트 소개 - Falcon 모델을 사용한 텍스트 생성

---

이 노트북에서는 세이지메이커 파이썬 SDK를 사용하여 Falcon 모델을 배포하고 텍스트 생성을 위해 미세 조정하는 방법을 실습합니다. 추론, 코드 생성, 질문 응답, 번역 등 여러 가지 예제 사용 사례를 보여줍니다. 미세 조정은 명령어 기반 미세 조정과 도메인 적응 미세 조정 두 가지 유형을 실습합니다. 

Falcon 모델은 [RefinedWeb 데이터 세트](https://huggingface.co/datasets/tiiuae/falcon-refinedweb)에서 훈련된 [아파치 2.0](https://jumpstart-cache-prod-us-east-2.s3.us-east-2.amazonaws.com/licenses/Apache-License/LICENSE-2.0.txt) 라이선스를 따르는 오픈 소스 모델입니다.

---

실습 목차

1. [Falcon 모델을 추론에 배포하기](#1-falcon-모델을-추론에-배포하기)
   * [1.1. 인스턴스 유형 변경](#11-인스턴스-유형-변경)
   * [1.2. GPU 수 변경](#12-gpu-수-변경하기)
   * [1.3. 모델 정보](#13-모델-정보)
   * [1.4. 지원되는 매개변수](#14-지원되는-매개변수)
2. [명령어 기반 미세 조정](#2-명령어-기반-미세-조정)
   * [2.1. 훈련 데이터 준비하기](#21-훈련-데이터-준비하기)
   * [2.2. 훈련 매개변수 준비하기](#22-훈련-매개변수-준비하기)
   * [2.3. 훈련 시작하기](#23-훈련-시작하기)
   * [2.4. 추론 엔드포인트 배포하기](#24-추론-엔드포인트-배포하기)
   * [2.5. 추론 쿼리 실행 및 모델 성능 비교](#25-추론-쿼리-실행-및-모델-성능-비교)
   * [2.6. 엔드포인트 정리하기](#26-엔드포인트-정리하기)
3. [도메인 적응 미세 조정](#3-도메인-적응-미세-조정)
   * [3.1. 훈련 데이터 준비하기](#31-훈련-데이터-준비하기)
   * [3.2. 훈련 매개변수 준비하기](#32-훈련-매개변수-준비하기)
   * [3.3. 훈련 시작하기](#33-훈련-시작하기)
   * [3.4. 추론 엔드포인트 배포하기](#34-추론-엔드포인트-배포하기)
   * [3.5. 추론 쿼리 실행 및 모델 성능 비교](#35-추론-쿼리-실행-및-모델-성능-비교)
   * [3.6. 엔드포인트 정리하기](#36-엔드포인트-정리하기)

## 1. Falcon 모델을 추론에 배포하기

In [ ]:
!pip install sagemaker --quiet --upgrade --force-reinstall
!pip install ipywidgets==7.0.0 --quiet

In [ ]:
model_id, model_version = "huggingface-llm-falcon-40b-instruct-bf16", "*"

In [ ]:
%%time
from sagemaker.jumpstart.model import JumpStartModel

my_model = JumpStartModel(model_id=model_id)
predictor = my_model.deploy()

### 1.1. 인스턴스 유형 변경

---

모델은 다음 인스턴스 유형에서 실습할 수 있습니다:

 - Falcon 7B 및 7B 명령어 모델: `ml.g5.2xlarge`, `ml.g5.4xlarge`, `ml.g5.8xlarge`, `ml.g5.16xlarge`, `ml.g5.12xlarge`, `ml.g5.24xlarge`, `ml.g5.48xlarge`, `ml.p4d.24xlarge`
 - Falcon 40B 및 40B 명령어 모델: `ml.g5.12xlarge`, `ml.g5.48xlarge`, `ml.p4d.24xlarge`

만약 해당 지역에 사용 가능한 인스턴스 유형이 없다면, 다른 인스턴스를 시도해 보세요. JumpStartModel 클래스에서 인스턴스 유형을 지정하여 이를 수행할 수 있습니다.

`my_model = JumpStartModel(model_id="huggingface-llm-falcon-40b-instruct-bf16", instance_type="ml.g5.12xlarge")`

---

### 1.2. GPU 수 변경하기

---

Falcon 모델은 허깅페이스 LLM DLC를 사용하여 제공되며, 모델 배포 시 GPU 수를 지정해야 합니다.

**Falcon 7B 및 7B 명령어 모델:** 허깅페이스 LLM DLC는 현재 7B 모델에 대해 샤딩을 지원하지 않습니다. 따라서 인스턴스에 여러 GPU가 있더라도 GPU 수를 늘리지 마세요.

**Falcon 40B 및 40B 명령어 모델:** 기본적으로 GPU 수는 4로 설정되어 있습니다. 그러나 `ml.g5.48xlarge` 또는 `ml.p4d.24xlarge`를 사용하는 경우, GPU 수를 8로 늘릴 수 있습니다. 다음과 같이 설정하세요:

`my_model = JumpStartModel(model_id="huggingface-llm-falcon-40b-instruct-bf16", instance_type="ml.g5.48xlarge")`

`my_model.env['SM_NUM_GPUS'] = '8'`

`predictor = my_model.deploy()`

---

In [ ]:
%%time

prompt = "Tell me about Amazon SageMaker."

payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.8,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "</s>"]
    }
}

response = predictor.predict(payload)
print(response[0]["generated_text"])

### 1.3. 모델 정보

---

Falcon은 [기술 혁신 연구소(Technology Innovation Institute, TII)](https://www.tii.ae/)에서 구축한 인과 디코더 전용 모델로, 정제된 웹 데이터(RefinedWeb)와 선별된 코퍼스를 사용하여 1조 개 이상의 토큰으로 훈련되었습니다. 이 모델은 아마존 세이지메이커에서 데이터 전처리 및 모델 훈련을 위해 맞춤 제작된 도구를 사용하여 구축되었습니다. 2023년 6월 6일 기준으로 사용 가능한 최고의 오픈 소스 모델입니다. Falcon-40B는 Llama, StableLM, RedPajama, MPT 등을 능가합니다. 비교를 보려면 [OpenLLM 리더보드](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)를 참조하세요. 이 모델은 FlashAttention과 multiquery를 특징으로 하여 추론에 최적화된 아키텍처를 제공합니다.

[Refined Web Dataset](https://huggingface.co/datasets/tiiuae/falcon-refinedweb): Falcon RefinedWeb is a massive English web dataset built by TII and released under an Apache 2.0 license. It is a highly filtered dataset with large scale de-duplication of CommonCrawl. It is observed that models trained on RefinedWeb achieve performance equal to or better than performance achieved by training model on curated datasets, while only relying on web data.

[Refined Web 데이터세트](https://huggingface.co/datasets/tiiuae/falcon-refinedweb): Falcon RefinedWeb은 TII에서 구축하고 아파치 2.0 라이선스로 공개한 방대한 영어 웹 데이터 세트입니다. 이 데이터세트는 CommonCrawl의 대규모 중복 제거를 포함해서 고도로 필터링된 데이터세트입니다. RefinedWeb에서 훈련된 모델은 선별된 데이터 세트에서 훈련된 모델과 동등하거나 더 나은 성능을 달성하는 것으로 관찰됩니다.

**모델 크기:**
- **Falcon-7b**: 70억 개의 파라미터를 가진 모델로 1.5조 개의 토큰으로 훈련되었습니다. 이는 유사한 오픈 소스 모델(MPT-7B, StableLM, RedPajama 등)보다 뛰어납니다. 비교를 해보려면 [OpenLLM 리더보드](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)를 참조하세요. 이 모델을 사용하려면 위의 셀에서 `model_id`를 "huggingface-llm-falcon-7b-bf16"으로 수정하세요.
- **Falcon-40B**: 400억 개의 파라미터를 가진 모델로 1조 개의 토큰으로 훈련되었습니다. 이 모델은 허깅페이스가 관리하는 공개 리더보드에서 Llama-65B, StableLM, RedPajama 및 MPT와 같은 유명한 모델을 능가하며 특수한 미세 조정 없이도 뛰어난 성능을 보여줍니다. 비교를 해보려면 [OpenLLM 리더보드](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)를 참조하세요. 

**명령어 모델(Falcon-7b-instruct/Falcon-40B-instruct):** 명령어 모델은 대화 및 명령어 데이터 세트의 혼합을 사용하여 미세 조정된 기본 Falcon 모델입니다. 이들은 바로 사용할 수 있는 대화/명령어 모델입니다. 이 모델을 사용하려면 위의 셀에서 `model_id`를 "huggingface-textgeneration-falcon-7b-instruct-bf16" 또는 "huggingface-textgeneration-falcon-40b-instruct-bf16"으로 수정하세요.

[권장사항](https://huggingface.co/tiiuae/falcon-7b)에 따르면 명령어 모델은 미세 조정 없이 사용하고, 기본 모델은 특정 작업에 맞게 추가로 미세 조정하는 것이 좋습니다.

**제한 사항:**

- Falcon 모델은 대부분 영어 데이터로 훈련되어 다른 언어에 일반화되지 않을 수 있습니다. 
- Falcon은 온라인과 훈련 데이터에서 흔히 접하는 고정 관념과 편견을 가지고 있을 수 있습니다. 따라서 실제 사용 시 적절한 보호 조치를 마련하고 필요한 예방 조치를 취하는 것이 좋습니다. 이 모델은 대부분의 사용 사례에 대해 추가로 미세 조정이 필요한 원시의 사전 훈련된 모델입니다.

**역자) 이러한 제한 사항 때문에 아래 실습에서 영어 데이터로 미세 조정합니다.**

---

In [ ]:
def query_endpoint(payload):
    """엔드포인트에 쿼리하고 응답 출력"""
    response = predictor.predict(payload)
    print(f"\033[1m Input:\033[0m {payload['inputs']}")
    print(f"\033[1m Output:\033[0m {response[0]['generated_text']}")

In [ ]:
# 코드 생성
payload = {"inputs": "Write a program to compute factorial in python:", "parameters":{"max_new_tokens": 200}}
query_endpoint(payload)

In [ ]:
payload = {
    "inputs": "Building a website can be done in 10 simple steps:",
    "parameters":{
        "max_new_tokens": 110,
        "no_repeat_ngram_size": 3
        }
}
query_endpoint(payload)

In [ ]:
# 번역
payload = {
    "inputs": """Translate English to French:

    sea otter => loutre de mer

    peppermint => menthe poivrée

    plush girafe => girafe peluche

    cheese =>""",
    "parameters":{
        "max_new_tokens": 3
    }
}

query_endpoint(payload)

In [ ]:
# 감정 분석
payload = {
    "inputs": """"I hate it when my phone battery dies."
                Sentiment: Negative
                ###
                Tweet: "My day has been :+1:"
                Sentiment: Positive
                ###
                Tweet: "This is the link to the article"
                Sentiment: Neutral
                ###
                Tweet: "This new music video was incredibile"
                Sentiment:""",
    "parameters": {
        "max_new_tokens":2
    }
}
query_endpoint(payload)

In [ ]:
# 질문-응답
payload = {
    "inputs": "Could you remind me when was the C programming language invented?",
    "parameters":{
        "max_new_tokens": 50
    }
}
query_endpoint(payload)

In [ ]:
# 레시피 생성
payload = {"inputs": "What is the recipe for a delicious lemon cheesecake?", "parameters":{"max_new_tokens": 400}}
query_endpoint(payload)

In [ ]:
# 요약
payload = {
    "inputs":"""Starting today, the state-of-the-art Falcon 40B foundation model from Technology
    Innovation Institute (TII) is available on Amazon SageMaker JumpStart, SageMaker's machine learning (ML) hub
    that offers pre-trained models, built-in algorithms, and pre-built solution templates to help you quickly get
    started with ML. You can deploy and use this Falcon LLM with a few clicks in SageMaker Studio or
    programmatically through the SageMaker Python SDK.
    Falcon 40B is a 40-billion-parameter large language model (LLM) available under the Apache 2.0 license that
    ranked #1 in Hugging Face Open LLM leaderboard, which tracks, ranks, and evaluates LLMs across multiple
    benchmarks to identify top performing models. Since its release in May 2023, Falcon 40B has demonstrated
    exceptional performance without specialized fine-tuning. To make it easier for customers to access this
    state-of-the-art model, AWS has made Falcon 40B available to customers via Amazon SageMaker JumpStart.
    Now customers can quickly and easily deploy their own Falcon 40B model and customize it to fit their specific
    needs for applications such as translation, question answering, and summarizing information.
    Falcon 40B are generally available today through Amazon SageMaker JumpStart in US East (Ohio),
    US East (N. Virginia), US West (Oregon), Asia Pacific (Tokyo), Asia Pacific (Seoul), Asia Pacific (Mumbai),
    Europe (London), Europe (Frankfurt), Europe (Ireland), and Canada (Central),
    with availability in additional AWS Regions coming soon. To learn how to use this new feature,
    please see SageMaker JumpStart documentation, the Introduction to SageMaker JumpStart –
    Text Generation with Falcon LLMs example notebook, and the blog Technology Innovation Institute trainsthe
    state-of-the-art Falcon LLM 40B foundation model on Amazon SageMaker. Summarize the article above:""",
    "parameters":{
        "max_new_tokens":200
        }
    }
query_endpoint(payload)

### 1.4. 지원되는 매개변수

***
추론 수행시 지원되는 매개변수는 다음과 같습니다:

* **max_length:** 모델이 생성하는 텍스트의 길이(입력 컨텍스트 길이 포함)가 `max_length`에 도달할 때까지 텍스트를 생성합니다. 이 값은 양의 정수여야 합니다.
* **max_new_tokens:** 모델이 생성하는 텍스트의 길이(입력 컨텍스트 길이 제외)가 `max_new_tokens`에 도달할 때까지 텍스트를 생성합니다. 이 값은 양의 정수여야 합니다.
* **num_beams:** 탐욕적 탐색에 사용되는 빔의 수입니다. 이 값은 `num_return_sequences`보다 크거나 같은 정수여야 합니다.
* **no_repeat_ngram_size:** 모델은 출력 시퀀스에서 `no_repeat_ngram_size` 크기의 단어 시퀀스가 반복되지 않도록 보장합니다. 이 값은 1보다 큰 양의 정수여야 합니다.
* **temperature:** 출력의 무작위성을 제어합니다. 높은 temperature 값은 확률이 낮은 단어를 포함하는 출력을, 낮은 temperature 값은 확률이 높은 단어를 포함하는 출력을 생성합니다. `temperature`가 0이면 탐욕적 디코딩(greedy decoding)이 수행됩니다. 이 값은 양의 실수여야 합니다.
* **early_stopping:** True로 설정하면 모든 빔 가설이 문장 끝 토큰에 도달하면 텍스트 생성이 완료됩니다. 이 값은 불(boolean) 유형 이어야 합니다.
* **do_sample:** True로 설정하면 다음 단어를 확률에 따라 샘플링합니다. 이 값은 불(boolean) 유형 이어야 합니다.
* **top_k:** 텍스트 생성의 각 단계에서 가장 가능성이 높은 top_k 단어만 샘플링합니다. 이 값은 양의 정수여야 합니다.
* **top_p:** 텍스트 생성의 각 단계에서 누적 확률이 top_p인 가능한 최소 집합의 단어를 샘플링합니다. 이 값은 0과 1 사이의 실수여야 합니다.
* **return_full_text:** True로 설정하면, 입력 텍스트가 생성된 출력 텍스트의 일부가 됩니다. 이 값은 불(boolean) 유형 이어야 하며, 기본값은 False입니다.
* **stop**: 이 값은 문자열 목록이어야 합니다. 지정된 문자열 중 하나가 생성되면 텍스트 생성을 중지합니다.

엔드포인트를 호출할 때 위에 언급된 매개변수 중 일부를 사용할 수 있습니다.

더 많은 매개변수와 허깅페이스 LLM DLC에 대한 정보는 [이 문서](https://huggingface.co/blog/sagemaker-huggingface-llm#4-run-inference-and-chat-with-our-model)를 참조하세요.

***

## 2. 명령어 기반 미세 조정

이번에는 `huggingface-llm-falcon-7b-instruct-bf16` 모델을 새로운 작업에 맞춰 명령어로 미세 조정하는 방법을 설명합니다. [섹션 1.3 모델 정보](#13-모델-정보)에서 설명했듯이 **Falcon-7b-instruct** 및 **Falcon-40B-instruct** 모델은 대화 및 명령어 데이터 세트의 혼합으로 미세 조정된 기본 Falcon 모델입니다.

**이번 작업에서는 주어진 문맥을 바탕으로 해당 텍스트에서 `답할 수 없는 질문`을 생성하도록 모델에 요청합니다. 예제는 이 노트북의 추론 섹션에 나와 있습니다.**

### 2.1. 훈련 데이터 준비하기

우리는 SQuAD2.0 데이터 세트의 일부를 사용하여 감독된 미세 조정을 진행합니다. 이 데이터 세트에는 인간 주석자가 Wikipedia 기사에 대해 제기한 질문이 포함되어 있습니다. SQuAD2.0에는 답이 있는 질문 외에도 약 50,000개의 답할 수 없는 질문이 포함되었습니다. 이러한 질문은 그럴듯하지만, 기사의 내용으로는 직접적으로 답할 수 없습니다. 이번 작업에서는 답할 수 없는 질문만 사용합니다.

인용: @article{rajpurkar2018know, title={Know what you don't know: Unanswerable questions for SQuAD}, author={Rajpurkar, Pranav and Jia, Robin and Liang, Percy}, journal={arXiv preprint arXiv:1806.03822}, year={2018} }

라이선스: Creative Commons Attribution-ShareAlike License (CC BY-SA 4.0)

In [ ]:
import boto3
import sagemaker
import json

# 현재 지역, 역할 및 기본 버킷 가져오기
aws_region = boto3.Session().region_name
aws_role = sagemaker.session.Session().get_caller_identity_arn()
output_bucket = sagemaker.Session().default_bucket()

# 출력할 때 유용한 코드
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

print(f"{bold}aws_region:{unbold} {aws_region}")
print(f"{bold}aws_role:{unbold} {aws_role}")
print(f"{bold}output_bucket:{unbold} {output_bucket}")

In [ ]:
from sagemaker.s3 import S3Downloader

# SQuAD2.0의 train split을 사용
original_data_file = "train-v2.0.json"

# 데이터가 다음 버킷에 미러링 되어있음
original_data_location = (
    f"s3://sagemaker-example-files-prod-{aws_region}/datasets/text/squad2.0/{original_data_file}"
)
S3Downloader.download(original_data_location, ".")

The training data must be formatted in JSON lines (.jsonl) format, where each line is a dictionary representing a single data sample. All training data must be in a single folder, however it can be saved in multiple jsonl files. The .jsonl file extension is mandatory. The training folder can also contain a template.json file describing the input and output formats.

훈련 데이터는 JSON lines (.jsonl) 형식이어야 하고 각 줄은 단일 데이터 샘플을 나타내는 딕셔너리입니다. 모든 훈련 데이터는 하나의 폴더에 있어야 하지만 여러 개의 jsonl 파일로 저장될 수 있습니다. .jsonl 파일 확장자는 필수입니다. 또한 훈련 폴더에는 입력 및 출력 형식을 설명하는 template.json 파일이 포함될 수 있습니다.

템플릿 파일이 제공되지 않은 경우, 아래의 기본 템플릿이 사용됩니다:

```json
{
    "prompt": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{context}`,
    "completion": "{response}",
}
```

이 경우 훈련 데이터는 `instruction`, `context` 및 `response` 필드를 포함해야 합니다.

이번 실습에서는 기본 프롬프트 템플릿을 사용하는 대신 사용자 정의 템플릿을 사용하겠습니다(아래 참조).

In [ ]:
template = {
    "prompt": "Ask a question which is related to the following text, but cannot be answered based on the text. Text: {context}",
    "completion": "{question}",
}

with open("template.json", "w") as f:
    json.dump(template, f)

다음으로 SQuAD 2.0 데이터 세트의 형식을 다시 맞춰줍니다. 처리된 데이터는 `task-data.jsonl` 파일로 저장합니다. 위의 셀에서 정의된 프롬프트 템플릿을 기준으로 `task-data.jsonl` 파일의 각 항목에는 **`context`** 와 `question` 필드가 포함됩니다. 실습을 위해 훈련 예제 수를 2000개로 제한합니다.

In [ ]:
local_data_file = "task-data.jsonl"  # .jsonl 확장자를 가진 어떤 이름의 파일도 가능 

with open(original_data_file) as f:
    data = json.load(f)

def preprocess_data(local_data_file, data, num_maximum_example):
    num_example_idx = 0  
    with open(local_data_file, "w") as f:
        for article in data["data"]:
            for paragraph in article["paragraphs"]:
                # 주어진 단락에 대한 질문 반복
                for qas in paragraph["qas"]:
                    if qas["is_impossible"]:
                        # 질문은 관련이 있지만, 답할 수 없음
                        example = {"context": paragraph["context"], "question": qas["question"]}
                        json.dump(example, f)
                        f.write("\n")
                        num_example_idx += 1
                        if num_example_idx >= num_maximum_example:
                            return

preprocess_data(local_data_file=local_data_file, data=data, num_maximum_example=10000)

프롬프트 템플릿(`template.json`)과 훈련 데이터(`task-data.jsonl`)를 S3 버킷에 업로드합니다.

In [ ]:
from sagemaker.s3 import S3Uploader

training_dataset_s3_path = f"s3://{output_bucket}/train_data"
S3Uploader.upload(local_data_file, training_dataset_s3_path)
S3Uploader.upload("template.json", training_dataset_s3_path)
print(f"{bold}training data:{unbold} {training_dataset_s3_path}")

### 2.2. 훈련 매개변수 준비하기

In [ ]:
from sagemaker import hyperparameters

my_hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)
print(my_hyperparameters)

하이퍼파라미터 덮어쓰기

In [ ]:
my_hyperparameters["epoch"] = "2"
my_hyperparameters["per_device_train_batch_size"] = "2"
my_hyperparameters["gradient_accumulation_steps"] = "2"
my_hyperparameters["instruction_tuned"] = "True"
print(my_hyperparameters)

하이퍼파라미터 유효성 검사

In [ ]:
hyperparameters.validate(model_id=model_id, model_version=model_version, hyperparameters=my_hyperparameters)

### 2.3. 훈련 시작하기

참고: `load_best_model_at_end` 매개변수(훈련이 끝날 때 훈련 중 발견된 최고의 모델을 로드할지 여부. 이 옵션이 활성화되면, 최고의 체크포인트가 항상 저장됩니다)는 기본값으로 "True"로 설정되어 있습니다. 훈련 종료 시 최고의 모델 체크포인트를 로드할 때(최고의 모델 체크포인트를 저장하기 전에 허깅페이스가 로드합니다) 메모리 사용량에 대한 오버헤드가 발생하여 메모리 부족 오류가 발생할 수 있습니다.

`load_best_model_at_end`를 설정하려면 `ml.g5.48xlarge`를 사용하는 것이 좋습니다. 설정하지 않으려면 `ml.g5.12xlarge`를 사용하는 것이 좋습니다.

In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator

instruction_tuned_estimator = JumpStartEstimator(
    model_id=model_id,
    hyperparameters=my_hyperparameters,
    instance_type="ml.g5.48xlarge",
)
instruction_tuned_estimator.fit(
    {"train": training_dataset_s3_path}, logs=True
)

훈련 성능 메트릭 추출. 훈련 중에는 CloudWatch를 통해 훈련 손실(training loss)과 검증 정확도/손실(validation accuracy/loss) 같은 성능 메트릭을 확인할 수 있습니다. 이 메트릭을 가져와서 노트북 내에서 분석할 수도 있습니다.

In [ ]:
from sagemaker import TrainingJobAnalytics

training_job_name = instruction_tuned_estimator.latest_training_job.job_name

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()
df.head(10)

### 2.4. 추론 엔드포인트 배포하기

In [ ]:
instruction_tuned_predictor = instruction_tuned_estimator.deploy()

### 2.5. 추론 쿼리 실행 및 모델 성능 비교

`test_paragraphs` 변수에 나열된 세 가지 예제를 검토합니다. `prompt` 변수에 정의된 프롬프트는 모델에게 문맥을 바탕으로 질문을 하도록 요청하며 질문이 문맥으로부터 **답을 찾을 수 없도록** 만들어야 한다는 것을 명시합니다.

[섹션 1](#1-falcon-모델을-추론에-배포하기)에서 배포한 사전 훈련된 Falcon-7b-instruct 모델(`huggingface-llm-falcon-7b-instruct-bf16`)과 이번 섹션에서 미세 조정한 Falcon-7b-instruct 모델의 성능을 비교합니다.

In [ ]:
prompt = "Ask a question which is related to the following text, but cannot be answered based on the text. Text: {context}"

# 출처: 위키피디아(Wikipedia), AWS 문서
test_paragraphs = [
    """
Adelaide is the capital city of South Australia, the state's largest city and the fifth-most populous city in Australia. "Adelaide" may refer to either Greater Adelaide (including the Adelaide Hills) or the Adelaide city centre. The demonym Adelaidean is used to denote the city and the residents of Adelaide. The Traditional Owners of the Adelaide region are the Kaurna people. The area of the city centre and surrounding parklands is called Tarndanya in the Kaurna language.
Adelaide is situated on the Adelaide Plains north of the Fleurieu Peninsula, between the Gulf St Vincent in the west and the Mount Lofty Ranges in the east. Its metropolitan area extends 20 km (12 mi) from the coast to the foothills of the Mount Lofty Ranges, and stretches 96 km (60 mi) from Gawler in the north to Sellicks Beach in the south.
""",
    """
Amazon Elastic Block Store (Amazon EBS) provides block level storage volumes for use with EC2 instances. EBS volumes behave like raw, unformatted block devices. You can mount these volumes as devices on your instances. EBS volumes that are attached to an instance are exposed as storage volumes that persist independently from the life of the instance. You can create a file system on top of these volumes, or use them in any way you would use a block device (such as a hard drive). You can dynamically change the configuration of a volume attached to an instance.
We recommend Amazon EBS for data that must be quickly accessible and requires long-term persistence. EBS volumes are particularly well-suited for use as the primary storage for file systems, databases, or for any applications that require fine granular updates and access to raw, unformatted, block-level storage. Amazon EBS is well suited to both database-style applications that rely on random reads and writes, and to throughput-intensive applications that perform long, continuous reads and writes.
""",
    """
Amazon Comprehend uses natural language processing (NLP) to extract insights about the content of documents. It develops insights by recognizing the entities, key phrases, language, sentiments, and other common elements in a document. Use Amazon Comprehend to create new products based on understanding the structure of documents. For example, using Amazon Comprehend you can search social networking feeds for mentions of products or scan an entire document repository for key phrases. 
You can access Amazon Comprehend document analysis capabilities using the Amazon Comprehend console or using the Amazon Comprehend APIs. You can run real-time analysis for small workloads or you can start asynchronous analysis jobs for large document sets. You can use the pre-trained models that Amazon Comprehend provides, or you can train your own custom models for classification and entity recognition. 
All of the Amazon Comprehend features accept UTF-8 text documents as the input. In addition, custom classification and custom entity recognition accept image files, PDF files, and Word files as input. 
Amazon Comprehend can examine and analyze documents in a variety of languages, depending on the specific feature. For more information, see Languages supported in Amazon Comprehend. Amazon Comprehend's Dominant language capability can examine documents and determine the dominant language for a far wider selection of languages.
""",
]

In [ ]:
parameters = {
    "max_new_tokens": 50,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 0.01,    
}

def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response

def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read())
    return model_predictions[0]["generated_text"]

def generate_question(endpoint_name, text):
    expanded_prompt = prompt.replace("{context}", text)
    payload = {"inputs": expanded_prompt, "parameters": parameters}
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name)
    generated_texts = parse_response(query_response)
    print(f"Response: {generated_texts}{newline}")

In [ ]:
print(f"{bold}Prompt:{unbold} {repr(prompt)}")
for paragraph in test_paragraphs:
    print("-" * 80)
    print(paragraph)
    print("-" * 80)
    print(f"{bold}pre-trained{unbold}")
    generate_question(predictor.endpoint_name, paragraph)
    print(f"{bold}fine-tuned{unbold}")
    generate_question(instruction_tuned_predictor.endpoint_name, paragraph)

사전 훈련된 모델은 특정하게 답할 수 없는 질문을 생성하도록 훈련되지 않았습니다. 입력 프롬프트에도 불구하고 텍스트에서 답할 수 있는 질문을 생성하는 경향이 있습니다. 미세 조정된 모델은 일반적으로 이 작업에서 더 나은 성능을 보이며 특히 더 큰 모델일수록 개선이 더 두드러집니다.

### 2.6. 엔드포인트 정리하기

In [ ]:
# 세이지메이커 엔드포인트 삭제
predictor.delete_model()
predictor.delete_endpoint()
instruction_tuned_predictor.delete_model()
instruction_tuned_predictor.delete_endpoint()

## 3. 도메인 적응 미세 조정

Falcon 모델은 도메인 적응 미세 조정도 지원합니다. 명령어 기반 미세 조정과 달리 명령어 형식의 데이터 세트를 준비할 필요 없이 아래와 같이 비구조화된 텍스트 문서를 직접 사용할 수 있습니다. 하지만 도메인 적응 미세 조정된 모델은 훈련 데이터 형식에 대한 제한이 적기 때문에 명령어 기반으로 미세 조정된 모델만큼 간결한 응답을 제공하지 않을 수 있습니다.

이번 실습에서는 Falcon 텍스트 생성 모델 `huggingface-llm-falcon-7b-bf16`을 사용합니다. 이는 Falcon 7B 모델의 명령어 기반 미세 조정 버전이 아닙니다. `huggingface-llm-falcon-7b-instruct-bf16`과 같은 명령어 기반으로 미세 조정된 모델 위에 도메인 적응 미세 조정을 수행할 수도 있지만 일반적으로는 권장하지 않습니다.

In [ ]:
model_id = "huggingface-llm-falcon-7b-bf16"

미국 증권거래위원회(SEC) 문서에서 금융 텍스트를 사용하여 `huggingface-llm-falcon-7b-bf16`을 금융 응용 프로그램에 맞게 미세 조정할 것입니다. 

다음은 훈련 및 검증 데이터에 대한 요구 사항입니다.

- **입력**: 훈련용, 선택적인 검증용 디렉토리. 각 디렉토리에는 CSV/JSON/TXT 파일이 포함됩니다.
    - CSV/JSON 파일의 경우 'text'라는 열이 있으면 해당 열의 데이터를 없으면 첫 번째 열의 데이터를 사용합니다.
    - 훈련, 검증 디렉토리(제공된 경우)에는 각각 하나의 파일만 있어야 합니다.
- **출력**: 추론을 위해 배포할 수 있는 훈련된 모델.

아래는 텍스트 생성 모델을 미세 조정하기 위한 TXT 파일의 예입니다. 이 TXT 파일은 2021년부터 2022년까지 아마존의 SEC 문서입니다.

---
```
This report includes estimates, projections, statements relating to our
business plans, objectives, and expected operating results that are “forward-
looking statements” within the meaning of the Private Securities Litigation
Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E
of the Securities Exchange Act of 1934. Forward-looking statements may appear
throughout this report, including the following sections: “Business” (Part I,
Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K),
and “Management’s Discussion and Analysis of Financial Condition and Results
of Operations” (Part II, Item 7 of this Form 10-K). These forward-looking
statements generally are identified by the words “believe,” “project,”
“expect,” “anticipate,” “estimate,” “intend,” “strategy,” “future,”
“opportunity,” “plan,” “may,” “should,” “will,” “would,” “will be,” “will
continue,” “will likely result,” and similar expressions. Forward-looking
statements are based on current expectations and assumptions that are subject
to risks and uncertainties that may cause actual results to differ materially.
We describe risks and uncertainties that could cause actual results and events
to differ materially in “Risk Factors,” “Management’s Discussion and Analysis
of Financial Condition and Results of Operations,” and “Quantitative and
Qualitative Disclosures about Market Risk” (Part II, Item 7A of this Form
10-K). Readers are cautioned not to place undue reliance on forward-looking
statements, which speak only as of the date they are made. We undertake no
obligation to update or revise publicly any forward-looking statements,
whether because of new information, future events, or otherwise.

...
```
---

아마존의 SEC 문서는 공개적으로 이용 가능한 [EDGAR](https://www.sec.gov/edgar/searchedgar/companysearch)에서 다운로드했습니다. 데이터에 접근하는 방법은 [여기](https://www.sec.gov/os/accessing-edgar-data)를 참조하세요.

### 3.1. 훈련 데이터 준비하기

아마존의 SEC 문서 훈련 데이터는 S3 버킷에 이미 저장되어 있습니다.

In [ ]:
from sagemaker.jumpstart.utils import get_jumpstart_content_bucket

# 샘플 훈련 데이터는 아래 버킷에서 사용할 수 있습니다
data_bucket = get_jumpstart_content_bucket(aws_region)
data_prefix = "training-datasets/sec_data"

training_dataset_s3_path = f"s3://{data_bucket}/{data_prefix}/train/"
validation_dataset_s3_path = f"s3://{data_bucket}/{data_prefix}/validation/"

### 3.2. 훈련 매개변수 준비하기

In [ ]:
from sagemaker import hyperparameters

my_hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

my_hyperparameters["epoch"] = "3"
my_hyperparameters["per_device_train_batch_size"] = "2"
my_hyperparameters["instruction_tuned"] = "False"
print(my_hyperparameters)

하이퍼파라미터 유효성 검사

In [ ]:
hyperparameters.validate(model_id=model_id, model_version=model_version, hyperparameters=my_hyperparameters)

### 3.3. 훈련 시작하기

In [ ]:
from sagemaker.jumpstart.estimator import JumpStartEstimator

domain_adaptation_estimator = JumpStartEstimator(
    model_id=model_id,
    hyperparameters=my_hyperparameters,
    instance_type="ml.p3dn.24xlarge",
)
domain_adaptation_estimator.fit(
    {"train": training_dataset_s3_path, "validation": validation_dataset_s3_path}, logs=True
)

훈련 성능 메트릭 추출. CloudWatch를 통해 훈련 중 훈련 손실(training loss)과 검증 정확도/손실(validation accuracy/loss) 같은 성능 메트릭을 확인할 수 있습니다. 이 메트릭을 가져와서 노트북 내에서 분석할 수도 있습니다.

In [ ]:
from sagemaker import TrainingJobAnalytics

training_job_name = domain_adaptation_estimator.latest_training_job.job_name

df = TrainingJobAnalytics(training_job_name=training_job_name).dataframe()
df.head(10)

### 3.4. 추론 엔드포인트 배포하기

도메인 적응 미세 조정된 모델과 사전 훈련된 모델을 각각 배포하고 그 성능을 비교합니다.

먼저 도메인 적응 미세조정된 모델을 배포합니다.

In [ ]:
domain_adaptation_predictor = domain_adaptation_estimator.deploy()

다음으로 사전 훈련된 `huggingface-llm-falcon-7b-bf16`을 배포합니다.

In [ ]:
my_model = JumpStartModel(model_id=model_id)
pretrained_predictor = my_model.deploy()

### 3.5. 추론 쿼리 실행 및 모델 성능 비교

In [ ]:
parameters = {
    "max_new_tokens": 300,
    "top_k": 50,
    "top_p": 0.8,
    "do_sample": True,
    "temperature": 1,
}

def generate_response(endpoint_name, text):
    payload = {"inputs": f"{text}:", "parameters": parameters}
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name)
    generated_texts = parse_response(query_response)
    print(f"Response: {generated_texts}{newline}")

In [ ]:
test_paragraph_domain_adaption = [
    "This Form 10-K report shows that",
    "We serve consumers through",
    "Our vision is",
]


for paragraph in test_paragraph_domain_adaption:
    print("-" * 80)
    print(paragraph)
    print("-" * 80)
    print(f"{bold}pre-trained{unbold}")
    generate_response(pretrained_predictor.endpoint_name, paragraph)
    print(f"{bold}fine-tuned{unbold}")
    generate_response(domain_adaptation_predictor.endpoint_name, paragraph)

미세 조정된 모델은 아마존의 SEC 보고서와 관련된 데이터 도메인에 더 구체적인 응답을 생성하기 시작합니다.

### 3.6. 엔드포인트 정리하기

In [ ]:
# # 세이지메이커 엔드포인트 삭제
# pretrained_predictor.delete_model()
# pretrained_predictor.delete_endpoint()
# domain_adaptation_predictor.delete_model()
# domain_adaptation_predictor.delete_endpoint()